- 환경 변수 가져오기

In [2]:
from dotenv import load_dotenv	# .env 파일 읽어주는 함수 load_dotenv 불러오기
import os		# os 환경변수에 접근
load_dotenv()		# .env 파일 내부 값을 환경변수로 등록

True

In [3]:
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_API_KEY

'pcsk_3JPLsi_KXPfeHMN4cGmKqcBVLWn9PbMN1tfnmkakiM7jZckEvJKcg2MeEC8g6ofMgars9a'

- Pinecone 클라이언트 초기화

In [4]:
from pinecone import Pinecone,ServerlessSpec		# Pinecone 라이브러리에서 Pinecone 객체와 ServerlessSpec(서버 설정용) 불러오기
pine = Pinecone(api_key=PINECONE_API_KEY)
# pine = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
# pinecone : 벡터를 저장하고 비교하는데 최적화된 DB.

# => Pinecone 서버에 로그인하고, 벡터DB 쓸 준비

- 인덱스 생성(서버리스)

In [5]:
index_name = 'wiki'		# Pinecone 안에 폴더(index/데이터 저장소)를 생성

In [6]:
pine.create_index(
	name=index_name,
	dimension=1536,						# : 벡터의 길이(차원 수)
	metric="cosine",					# 벡터들끼리 얼마나 비슷한지. (코사인 유사도) 의미가 비슷할 수록 각도가 작음.
	# -> 대부분의 AI 임베딩 모델은 cosine 사용. (두 문장의 의미적 유사도를 비교하기에 적절)
	spec=ServerlessSpec(cloud='aws',region='us-east-1')		# 서버가 어디에 만들어질지 설정
	# ㄴ cloud 어떤 클라우드에 저장할지, region 어느 지역 서버를 쓸지. (aws/us-east-1 : 기본설정. 가장 안정적)
)

# => 의미가 비슷한 데이터끼리 잘 찾게 해주고, AWS 서버에 저장해라

# "status": {
#        "ready": true,
#        "state": "Ready"					: 인덱스가 잘 만들어졌다는 의미

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '649e59db00d56e908cc0329b4e245a0f', 'date': 'Fri, 14 Nov 2025 06:46:34 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


- 임베딩 객체 생성

In [7]:
# 사용할 인덱스 가져오기
index = pine.Index(index_name)
index.describe_index_stats()				# Pinecone 인덱스의 상태 요약
# ㄴ stats : statistics(통계,요약)

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [8]:
from langchain_openai import OpenAIEmbeddings
enbedding = OpenAIEmbeddings(model='text-embedding-3-small')	
# 임베딩 모델의 차원(dimension) 과 생성한 인덱스의 dimension 이 같아야 합니다.
# 			ㄴ : 이 임베딩 모델이 만드는 벡터의 길이가 Pinecone 인덱스의 dimension=1536 설정과 같아야 함

# 임베딩(embedding) : 단어/문장 같은 글자를 숫자 벡터로 바꿔주는 기술(글의 의미를 숫자로 표현하는 방법)
# 			ㄴ AI 검색, 챗봇 기억 기능, 추천 시스템에 사용

- 벡터 DB 에 저장할 데이터셋 가져오기
	+ 영어로 20231101 버전의 위키백과 600만개의 row 중 100개만 가져오기
	+ 데이터의 크기가 크면 임베딩 시간이 오래걸림.

In [ ]:
from datasets import load_dataset					# load_dataset() : 파이썬 안에서 허깅페이스 서버에 자동 연결해서 데이터를 다운로드 함.
# Hugging Face : AI 모델/데이터 저장소(API 제공)
# pip install datasets
dataset = load_dataset("wikimedia/wikipedia", "20231101.en", split="train[:100]")		# 7GB 전체를 다운로드
dataset.to_parquet("train-03.parquet")
dataset = load_dataset("parquet", data_files=["train-03.parquet"])

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 23.98ba/s]
Generating train split: 100 examples [00:00, 8326.16 examples/s]


In [ ]:
print(dataset)				# num_rows : 데이터 갯수

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 100
    })
})


In [ ]:
data = dataset['train'][0:100]			# 여전히 dict
print(len(data))				# dict : 4개의 키값. 4개의 키 안에 리스트 크기 100
print(data['text'][0])	# text 컬럼에서 첫번째(row index 0) 데이터를 출력한 것 => 문서 1개를 불러온것
# select() 는 dataset 함수
# 	ㄴ : Hugging Face의 datasets 패키지에서 만든 데이터처리 함수. 특정 인덱스(rows)만 골라 새로운 dataset을 만드는 함수. 
data = dataset['train'].select(range(100))			# dict 의 리스트.	range(): 연속된 숫자들을 만들어주는 도구(함수).			=> dataset 구조로 변환
print(len(data))		# list 의 크기 : 100

4
Anarchism is a political philosophy and movement that is skeptical of all justifications for authority and seeks to abolish the institutions it claims maintain unnecessary coercion and hierarchy, typically including nation-states, and capitalism. Anarchism advocates for the replacement of the state with stateless societies and voluntary free associations. As a historically left-wing movement, this reading of anarchism is placed on the farthest left of the political spectrum, usually described as the libertarian wing of the socialist movement (libertarian socialism).

Humans have lived in societies without formal hierarchies long before the establishment of states, realms, or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose. Although traces of anarchist ideas are found all throughout history, modern anarchism emerged from the Enlightenment. During the latter half of the 19th and the first decades of the 20th century, the anarchist movement

In [18]:
# print(data['text][0])
print(data[85])

{'id': '711', 'url': 'https://en.wikipedia.org/wiki/Albert%20Sidney%20Johnston', 'title': 'Albert Sidney Johnston', 'text': 'Albert Sidney Johnston (February 2, 1803 – April 6, 1862) served as a general in three different armies: the Texian Army, the United States Army, and the Confederate States Army. He saw extensive combat during his 34-year military career, fighting actions in the Black Hawk War, the Texas-Indian Wars, the Mexican–American War, the Utah War, and the American Civil War.\n\nConsidered by Confederate States President Jefferson Davis to be the finest general officer in the Confederacy before the later emergence of Robert E. Lee, he was killed early in the Civil War at the Battle of Shiloh on April 6, 1862. Johnston was the highest-ranking officer on either side killed during the entire war. Davis believed the loss of General Johnston "was the turning point of our fate."\n\nJohnston was unrelated to Confederate general Joseph E. Johnston.\n\nEarly life and education\nJo

- 청크
	+ : 긴 문서를 잘게 잘라 나눈 "조각"
	+ splitter 객체를 생성해서 문자열을 나누기
	+ RAG : 검색 기반 생성 AI (llm => 검색해서 자료수집 -> 자료 참고하여 답변 생성)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
	chunk_size=400,							# 텍스트 분할크기(청크 1개의 최대 길이)
	chunk_overlap=20,						# 분할할 텍스트의 중첩 크기
	length_function=len,
	separators=['\n','']				# 분할할때 사용할 텍스트(단어) 구분자
)